In [1]:
import sys
sys.path.append('../')
from tqdm.auto import tqdm
from loader import get_dataloader
from augmentations import get_composed_augmentations
import torch

from tqdm.notebook import tqdm

from torch.nn.functional import softmax

import sys
sys.path.append('../')
from utils import roc_btw_arr

In [9]:
import numpy as np

In [2]:
flowers_dl = get_dataloader({'dataset': 'Flowers', 
                           'split':'evaluation', 
                           'path': '../datasets', 
                           'batch_size': 64, 
                           'n_workers': 2, 
                           'shuffle':False})

Using No Augmentations
Using No Augmentations


In [14]:
import matplotlib.pyplot as plt

In [7]:
ds = flowers_dl.dataset

In [33]:
ds.indices

array([6734, 6735, 6737, ..., 8044, 8045, 8047], dtype=uint16)

In [34]:
len(ds.indices)

6149

In [4]:
a = torch.load('../results/RImgNet/vit_hf_md/OOD_rank_Flowers.pkl')

In [5]:
a

array([10122, 10109, 10090, ..., 10056, 10037, 10104])

In [10]:
sorted_idx = np.argsort(a)

In [11]:
sorted_idx

array([2105,  548, 5521, ..., 1902,  925,  825])

In [39]:
vis_img = torch.stack([ds[i][0] for i in sorted_idx[:64]
                      if i not in {sorted_idx[j] for j in [0,1,2,3,4,6,7,10]}])

In [102]:
l_insects = [2105, 548, 5521, 6062, 3856, 798, 2125, 2192, 544, 912, 2122, 6057, 2166, 321, 2175, 2278, 2761, 2232, 2110, 5870,
            917, 5816, 2191, 2174, 2705, 5709, 4136, 610, 2167, 5515, 808, 3606, 2196, 3124, 1080, 487, 602, 462, 4118]

In [116]:
l_insect_indices = ds.indices[l_insects]

In [121]:
torch.save(l_insect_indices, 'remove_flower_index.pkl')

In [114]:
import shutil

In [115]:
! mkdir -p ../datasets/flowers_clean/jpg

In [118]:
len(ds.indices)

6149

In [119]:
len(l_insects)

39

In [120]:
6149 - 39

6110

In [117]:
for idx in ds.indices:
    if idx not in l_insect_indices:
        shutil.copy(f'../datasets/flowers/jpg/image_{idx:05d}.jpg', f'../datasets/flowers_clean/jpg/image_{idx:05d}.jpg')

In [ ]:
[0,1,2,3,4,6,7,10]

In [28]:
from torchvision.utils import make_grid

In [122]:
fgvc_dl = get_dataloader({'dataset': 'FGVC', 
                           'split':'evaluation', 
                           'path': '../datasets', 
                           'batch_size': 64, 
                           'n_workers': 2, 
                           'shuffle':False})

Using No Augmentations
Using No Augmentations


In [123]:
ds = fgvc_dl.dataset

In [126]:
! mkdir ../datasets/FGVC_test_gan

In [127]:
import os

In [128]:
os.path.basename(file)

NameError: name 'file' is not defined

In [131]:
for file, _ in ds.samples:
    shutil.copy(file, '../datasets/FGVC_test_gan/')